# Thư viện

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.metrics import mean_squared_error as mse
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Hàm

In [ ]:
def df_to_X_y(df, window_size = 5, type = 1):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np) - window_size):
    row = [r for r in df_as_np[i : i + window_size]]
    X.append(row)
    if type == 1:
      label = df_as_np[i + window_size]
    elif type == 2:
      label = df_as_np[i + window_size][0]
    elif type == 3:
      label = [df_as_np[i + window_size][0], df_as_np[i + window_size][1]]
    else:
      label = []
      for i in range(14):
        label.append(df_as_np[i + window_size][i])
    y.append(label)
  return np.array(X), np.array(y)

In [ ]:
def plot_predictions(model, X, y,):
  predictions = model.predict(X).flatten()
  actuals = pd.DataFrame(data={'Predictions':predictions, 'Actuals':y})
  plt.figure(figsize=(24, 10))
  plt.plot(actuals['Predictions'])
  plt.plot(actuals['Actuals'])
  return actuals, mse(y, predictions)

In [ ]:
def postprocess_temp(arr):
  arr = (arr * temp_training_std) + temp_training_mean
  return arr

def postprocess_p(arr):
  arr = (arr * p_training_std) + p_training_mean
  return arr

def plot_predictions_2(model, X, y):
  predictions = model.predict(X)
  p_preds, temp_preds = postprocess_p(predictions[:, 0]), postprocess_temp(predictions[:, 1])
  p_actuals, temp_actuals = postprocess_p(y[:, 0]), postprocess_temp(y[:, 1])
  df = pd.DataFrame(data={'Temperature Predictions': temp_preds,
                          'Temperature Actuals':temp_actuals,
                          'Pressure Predictions': p_preds,
                          'Pressure Actuals': p_actuals
                          })
  plt.plot(df['Temperature Predictions'])
  plt.plot(df['Temperature Actuals'])
  plt.plot(df['Pressure Predictions'])
  plt.plot(df['Pressure Actuals'])
  return df

In [ ]:
def preprocess(X):
  X[:, :, 0] = (X[:, :, 0] - temp_training_mean) / temp_training_std

In [ ]:
def preprocess_input(X):
  X[:, :, 0] = (X[:, :, 0] - p_training_mean) / p_training_std
  X[:, :, 1] = (X[:, :, 1] - temp_training_mean) / temp_training_std

def preprocess_output(y):
  y[:, 0] = (y[:, 0] - p_training_mean) / p_training_std
  y[:, 1] = (y[:, 1] - temp_training_mean) / temp_training_std

# Dữ liệu

## Tạo

In [ ]:
drive_path = 'drive/MyDrive/Internship/RNN/'
csv_path = drive_path + 'Data.csv'

df = pd.read_csv(csv_path)
df.index = pd.to_datetime(df['NgayGio'], format='%Y-%m-%d %H:%M:%S')
df = df.drop(['NgayGio'], axis=1)

## Xử lý

In [ ]:
for index, value in enumerate(df['ApSuat']):
  if value > 0:
    pass
  else:
    df['ApSuat'][index] = df['ApSuat'].mean()

for index, value in enumerate(df['TamNhin']):
  if value > 0:
    pass
  else:
    df['TamNhin'][index] = df['TamNhin'].mean()

for index, value in enumerate(df['ChiSoUV']):
  if value > 0:
    pass
  else:
    df['ChiSoUV'][index] = df['ChiSoUV'].mean()

In [ ]:
le = preprocessing.LabelEncoder()
df['TrangThaiMua'] = le.fit_transform(df['TrangThaiMua'])
df['KetLuan'] = le.fit_transform(df['KetLuan'])

In [ ]:
df['XacSuatMua'] = df['XacSuatMua'] / 100

# Thông số đào tạo

In [ ]:
EPOCHS = 30
LR = 1e-4
drive_path = 'drive/MyDrive/Internship/RNN'

# Đào tạo (1 đặc trưng)

## Đặt biến tạm (đặc trưng)

In [ ]:
temp = df['NhietDo']

## Tách

In [ ]:
WINDOW_SIZE = 5

X, y = df_to_X_y(temp, WINDOW_SIZE, 1)

n = len(X)

X_train, y_train = X[:int(n*0.8)], y[:int(n*0.8)]
X_val, y_val = X[int(n*0.8):int(n*0.9)], y[int(n*0.8):int(n*0.9)]
X_test, y_test = X[int(n*0.9):], y[int(n*0.9):]

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')
y_val = np.asarray(y_val).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')

## Mô hình LSTM

In [ ]:
model_path = drive_path + 'ModelLSTM/'
cp = ModelCheckpoint(model_path, save_best_only=True)

model = Sequential()
model.add(InputLayer((5, 1)))
model.add(LSTM(64, activation='relu'))
model.add(Dense(32, 'relu'))
model.add(Dense(64, 'relu'))
model.add(Dense(1, 'linear'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=LR), metrics=[RootMeanSquaredError()])

model.summary()

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[cp])

In [ ]:
model = load_model(model_path)
plot_predictions(model, X_test, y_test)

## Mô hình Conv1D

In [ ]:
model_path = drive_path + 'ModelConv1D/'
cp = ModelCheckpoint(model_path, save_best_only=True)

model = Sequential()
model.add(InputLayer((5, 1)))
model.add(Conv1D(64, kernel_size=2))
model.add(Flatten())
model.add(Dense(32, 'relu'))
model.add(Dense(64, 'relu'))
model.add(Dense(1, 'linear'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=LR), metrics=[RootMeanSquaredError()])

model.summary()

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[cp])

In [ ]:
model = load_model(model_path)
plot_predictions(model, X_test, y_test)

## Mô hình GRU

In [ ]:
model_path = drive_path + 'ModelGRU/'
cp = ModelCheckpoint(model_path, save_best_only=True)

model = Sequential()
model.add(InputLayer((5, 1)))
model.add(GRU(64))
model.add(Dense(32, 'relu'))
model.add(Dense(64, 'relu'))
model.add(Dense(1, 'linear'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=LR), metrics=[RootMeanSquaredError()])

model.summary()

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[cp])

In [ ]:
model = load_model(model_path)
plot_predictions(model, X_test, y_test)

# Đào tạo (1 đặc trưng & 4 đặc trưng thời gian)

## Đặt biến tạm

In [ ]:
day = 60*60*24
year = 365.2425*day

temp_df = pd.DataFrame({'NhietDo':temp})
temp_df['Giay'] = temp_df.index.map(pd.Timestamp.timestamp)

temp_df['NgaySin'] = np.sin(temp_df['Giay'] * (2 * np.pi / day))
temp_df['NgayCos'] = np.cos(temp_df['Giay'] * (2 * np.pi / day))
temp_df['NamSin'] = np.sin(temp_df['Giay'] * (2 * np.pi / year))
temp_df['NamCos'] = np.cos(temp_df['Giay'] * (2 * np.pi / year))

temp_df = temp_df.drop('Giay', axis=1)

## Tách

In [ ]:
WINDOW_SIZE = 6

X, y = df_to_X_y(temp_df, WINDOW_SIZE, 2)

n = len(X)

X_train, y_train = X[:int(n*0.8)], y[:int(n*0.8)]
X_val, y_val = X[int(n*0.8):int(n*0.9)], y[int(n*0.8):int(n*0.9)]
X_test, y_test = X[int(n*0.9):], y[int(n*0.9):]

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')
y_val = np.asarray(y_val).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')

## Chuẩn hóa

In [ ]:
temp_training_mean = np.mean(X_train[:, :, 0])
temp_training_std = np.std(X_train[:, :, 0])

In [ ]:
preprocess(X_train)
preprocess(X_val)
preprocess(X_test)

## Mô hình LSTM

In [ ]:
model_path = drive_path + 'ModelLSTM/'
cp = ModelCheckpoint(model_path, save_best_only=True)

model = Sequential()
model.add(InputLayer((6, 5)))
model.add(LSTM(64), activation='relu')
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'linear'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=LR), metrics=[RootMeanSquaredError()])

model.summary()

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[cp])

In [ ]:
model = load_model(model_path)
plot_predictions(model, X_test, y_test)

# Đào tạo (2 đặc trưng & 4 đặc trưng thời gian)

## Đặt biến tạm

In [ ]:
p_temp_df = pd.concat([df['DoAm'], temp_df], axis=1)

## Tách

In [ ]:
WINDOW_SIZE = 7

X, y = df_to_X_y(p_temp_df, WINDOW_SIZE, 3)

n = len(X)

X_train, y_train = X[:int(n*0.8)], y[:int(n*0.8)]
X_val, y_val = X[int(n*0.8):int(n*0.9)], y[int(n*0.8):int(n*0.9)]
X_test, y_test = X[int(n*0.9):], y[int(n*0.9):]

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')
y_val = np.asarray(y_val).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')

## Chuẩn hóa

In [ ]:
p_training_mean = np.mean(X_train[:, :, 0])
p_training_std = np.std(X_train[:, :, 0])

temp_training_mean = np.mean(X_train[:, :, 1])
temp_training_std = np.std(X_train[:, :, 1])

preprocess_input(X_train)
preprocess_input(X_val)
preprocess_input(X_test)

preprocess_output(y_train)
preprocess_output(y_val)
preprocess_output(y_test)

## Mô hình LSTM

In [ ]:
model_path = drive_path + 'ModelLSTM/'
cp = ModelCheckpoint(model_path, save_best_only=True)

model = Sequential()
model.add(InputLayer((7, 6)))
model.add(LSTM(64, activation='relu'))
model.add(Dense(8, 'relu'))
model.add(Dense(2, 'linear'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=LR), metrics=[RootMeanSquaredError()])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[cp])

In [ ]:
model_path = drive_path + 'ModelLSTM/'
model = load_model(model_path)
plot_predictions_2(model, X_test, y_test, 0, 100)

In [ ]:
post_processed_df = plot_predictions_2(model, X_test, y_test)
start, end = 0, 100

In [ ]:
plt.plot(post_processed_df['Temperature Predictions'][start:end])
plt.plot(post_processed_df['Temperature Actuals'][start:end])

In [ ]:
plt.plot(post_processed_df['Pressure Predictions'][start:end])
plt.plot(post_processed_df['Pressure Actuals'][start:end])

## Mô hình LSTM 2

In [ ]:
model_path = drive_path + 'ModelLSTM/'
cp = ModelCheckpoint(model_path, save_best_only=True)

model = Sequential()
model.add(InputLayer((7, 6)))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu'))
model.add(Dense(16, 'relu'))
model.add(Dense(2, 'linear'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=LR), metrics=[RootMeanSquaredError()])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[cp])

In [ ]:
model_path = drive_path + 'ModelLSTM/'
model = load_model(model_path)
plot_predictions_2(model, X_test, y_test)

In [ ]:
post_processed_df = plot_predictions_2(model, X_test, y_test)
start, end = 0, 100

In [ ]:
plt.plot(post_processed_df['Temperature Predictions'][start:end])
plt.plot(post_processed_df['Temperature Actuals'][start:end])

In [ ]:
plt.plot(post_processed_df['Pressure Predictions'][start:end])
plt.plot(post_processed_df['Pressure Actuals'][start:end])

## Mô hình Conv1D

In [ ]:
model_path = drive_path + 'ModelConv1D/'
cp = ModelCheckpoint(model_path, save_best_only=True)

model = Sequential()
model.add(InputLayer((7, 6)))
model.add(Conv1D(64, kernel_size=2, activation='relu'))
model.add(Flatten())
model.add(Dense(8, 'relu'))
model.add(Dense(2, 'linear'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=LR), metrics=[RootMeanSquaredError()])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[cp])

In [ ]:
model_path = drive_path + 'ModelConv1D/'
model = load_model(model_path)
plot_predictions_2(model, X_test, y_test)

In [ ]:
post_processed_df = plot_predictions_2(model, X_test, y_test)
start, end = 0, 100

In [ ]:
plt.plot(post_processed_df['Temperature Predictions'][start:end])
plt.plot(post_processed_df['Temperature Actuals'][start:end])

In [ ]:
plt.plot(post_processed_df['Pressure Predictions'][start:end])
plt.plot(post_processed_df['Pressure Actuals'][start:end])

# 14 đặc trưng

## Dữ liệu

In [ ]:
df['NgaySin'] = temp_df['NgaySin']
df['NgayCos'] = temp_df['NgayCos']
df['NamSin'] = temp_df['NamSin']
df['NamCos'] = temp_df['NamCos']

## Tách

In [ ]:
WINDOW_SIZE = 7

X, y = df_to_X_y(df, WINDOW_SIZE, 4)

n = len(X)

X_train, y_train = X[:int(n*0.8)], y[:int(n*0.8)]
X_val, y_val = X[int(n*0.8):int(n*0.9)], y[int(n*0.8):int(n*0.9)]
X_test, y_test = X[int(n*0.9):], y[int(n*0.9):]

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')
y_val = np.asarray(y_val).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')

## Chuẩn hóa

In [ ]:
for i in range(14):
   X_train[:, :, i] = (X_train[:, :, i] - np.mean(X_train[:, :, i])) / np.std(X_train[:, :, i])
   X_val[:, :, i] = (X_val[:, :, i] - np.mean(X_train[:, :, i])) / np.std(X_train[:, :, i])
   X_test[:, :, i] = (X_test[:, :, i] - np.mean(X_train[:, :, i])) / np.std(X_train[:, :, i])
   
   y_train[:, i] = (y_train[:, i] - np.mean(X_train[:, :, i])) / np.std(X_train[:, :, i])
   y_val[:, i] = (y_val[:, i] - np.mean(X_train[:, :, i])) / np.std(X_train[:, :, i])
   y_test[:, i] = (y_test[:, i] - np.mean(X_train[:, :, i])) / np.std(X_train[:, :, i])

## Mô hình

In [ ]:
model_path = drive_path + 'ModelConv1D/'
cp = ModelCheckpoint(model_path, save_best_only=True)

model = Sequential()
model.add(InputLayer((7, 18)))
model.add(Conv1D(64, kernel_size=2, activation='relu'))
model.add(Flatten())
model.add(Dense(16, 'relu'))
model.add(Dense(14, 'linear'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=LR), metrics=[RootMeanSquaredError()])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, callbacks=[cp])